In [11]:
import pickle
import pandas as pd
import geopandas as gpd

### 3.5职住人口关联分布数据

In [12]:
# 广东栅格中心点
with open(r'./data/input/gz_grid_wgs', 'rb') as f:
    grid_gdf = pickle.load(f)
id_list = [i for i in range(1, len(grid_gdf) + 1)]
grid_gdf

,grid_id,lon,lat,city,name,geometry
0,w7w773h,110.188065,20.264969,V0440800,NaN,POINT (110.18806 20.26497)
1,w7w77ur,110.214157,20.282822,V0440800,NaN,POINT (110.21416 20.28282)
2,w7whvyd,109.943619,20.558853,V0440800,NaN,POINT (109.94362 20.55885)
3,w7wjrqz,110.016403,20.648117,V0440800,NaN,POINT (110.01640 20.64812)
4,w7wkmgv,110.299301,20.455856,V0440800,NaN,POINT (110.29930 20.45586)
...,...,...,...,...,...,...
1389291,ws6d1yq,116.101456,24.292831,V0441400,NaN,POINT (116.10146 24.29283)
1389292,ws6sgsx,116.179733,24.766617,V0441400,NaN,POINT (116.17973 24.76662)
1389293,ws715tt,116.880112,24.112930,V0445100,NaN,POINT (116.88011 24.11293)
1389294,ws715v6,116.886978,24.111557,V0445100,NaN,POINT (116.88698 24.11156)


In [13]:
# 交通分区矢量
taz_gdf = gpd.read_file(r'./data/input/全交通分区_spatialjoin.shp')
taz_gdf.rename(columns={'name': 'city_name', 'district':'district_name', 'id':'taz_id'}, inplace=True)
taz_gdf

,taz_id,city_name,district_name,geometry
0,3001,东莞市,佛山外部,"POLYGON ((113.54401 23.06203, 113.54585 23.068..."
1,3002,东莞市,佛山外部,"POLYGON ((113.59893 22.91384, 113.61559 22.932..."
2,3003,东莞市,佛山外部,"POLYGON ((113.78617 22.93732, 113.78670 22.936..."
3,3004,东莞市,佛山外部,"POLYGON ((113.97796 22.99039, 113.97794 22.990..."
4,3005,东莞市,佛山外部,"POLYGON ((114.12707 23.00180, 114.12720 23.000..."
...,...,...,...,...
569,7007,肇庆市,佛山外部,"POLYGON ((112.06313 23.48404, 112.06771 23.481..."
570,7008,肇庆市,佛山外部,"POLYGON ((112.67355 23.29873, 112.67535 23.311..."
571,7009,肇庆市,佛山外部,"POLYGON ((112.68484 23.34429, 112.68685 23.347..."
572,7010,肇庆市,佛山外部,"POLYGON ((112.82725 23.46672, 112.82584 23.470..."


In [14]:
# 空间连接
grid_gdf = gpd.sjoin(grid_gdf, taz_gdf, how="inner", predicate='within')
if grid_gdf['taz_id'].dtypes == 'float64':
    grid_gdf['taz_id'] = grid_gdf['taz_id'].astype('int32')
# 去重
grid_gdf.drop_duplicates(subset=['grid_id'], inplace=True)
# 保留有用信息
grid_gdf = grid_gdf[['grid_id', 'lon', 'lat', 'geometry', 'taz_id', 'city_name', 'district_name']]
grid_gdf

,grid_id,lon,lat,geometry,taz_id,city_name,district_name
0,w7w773h,110.188065,20.264969,POINT (110.18806 20.26497),13000,湛江市,佛山外部
1,w7w77ur,110.214157,20.282822,POINT (110.21416 20.28282),13000,湛江市,佛山外部
2,w7whvyd,109.943619,20.558853,POINT (109.94362 20.55885),13000,湛江市,佛山外部
3,w7wjrqz,110.016403,20.648117,POINT (110.01640 20.64812),13000,湛江市,佛山外部
4,w7wkmgv,110.299301,20.455856,POINT (110.29930 20.45586),13000,湛江市,佛山外部
...,...,...,...,...,...,...,...
1325423,ws07x2v,113.177719,23.120041,POINT (113.17772 23.12004),1337,佛山市,南海区
1326765,ws07x2w,113.179092,23.118668,POINT (113.17909 23.11867),1337,佛山市,南海区
1328134,ws07x2x,113.180466,23.118668,POINT (113.18047 23.11867),1337,佛山市,南海区
1329489,ws07x2y,113.179092,23.120041,POINT (113.17909 23.12004),1337,佛山市,南海区


In [15]:
# 职住原始数据
home_work_df = pd.read_csv(
    r'./data/input/zc_v5_homework.csv', low_memory=False)
# 清洗职住数据缺失数据
home_work_df.dropna(subset=['home_grid', 'work_grid'], how='any', inplace=True)
home_work_df.drop(columns=['home_city', 'work_city'], inplace=True)
home_work_df

,home_grid,home_lon,home_lat,work_grid,work_lon,work_lat,count
178943,ws01f5b,112.588577,22.828903,ws01fh1,112.589951,22.830276,1
178944,ws04hct,112.716293,22.860489,ws01fh1,112.589951,22.830276,1
178945,ws0d0vx,113.246384,22.882462,ws03m87,113.098068,22.721786,1
178946,ws03xj2,113.159866,22.793198,ws03tgg,113.109055,22.784958,1
178948,ws03xkk,113.176346,22.787704,ws03tgg,113.109055,22.784958,1
...,...,...,...,...,...,...,...
1151901,ws0m367,112.911301,23.435898,ws0m36e,112.911301,23.437271,1
1151902,ws0m4xg,112.966232,23.422165,ws0m36e,112.911301,23.437271,1
1151903,ws0m33b,112.907181,23.433151,ws0m36e,112.911301,23.437271,1
1151904,ws0m6h1,112.941513,23.445511,ws0m3xq,112.926407,23.463364,1


In [16]:
# 字段连接，匹配职住数据与区划矢量数据
home_work_merge_df = pd.merge(home_work_df, grid_gdf, 
                              left_on='home_grid', right_on='grid_id',
                              how='inner')
home_work_merge_df.drop(columns=['grid_id', 'lon', 'lat','geometry'], inplace=True)
home_work_merge_df.rename(columns={'taz_id': 'home_small_zone', 'city_name':'home_city_name', 'district_name':'home_district_name'}, inplace=True)

home_work_merge_df = pd.merge(home_work_merge_df, grid_gdf, 
                              left_on='work_grid', right_on='grid_id',
                              how='inner')
home_work_merge_df.drop(columns=['grid_id', 'lon', 'lat','geometry'], inplace=True)
home_work_merge_df.rename(columns={'taz_id': 'work_small_zone', 'city_name':'work_city_name', 'district_name':'work_district_name'}, inplace=True)
home_work_merge_df

,home_grid,home_lon,home_lat,work_grid,work_lon,work_lat,count,home_small_zone,home_city_name,home_district_name,work_small_zone,work_city_name,work_district_name
0,ws01f5b,112.588577,22.828903,ws01fh1,112.589951,22.830276,1,1089,佛山市,高明区,1089,佛山市,高明区
1,ws04hct,112.716293,22.860489,ws01fh1,112.589951,22.830276,1,1089,佛山市,高明区,1089,佛山市,高明区
2,ws01f5b,112.588577,22.828903,ws01cgw,112.585831,22.827530,1,1089,佛山市,高明区,1089,佛山市,高明区
3,ws04r24,112.822037,22.896194,ws01cgw,112.585831,22.827530,1,1086,佛山市,高明区,1089,佛山市,高明区
4,ws04hct,112.716293,22.860489,ws045e2,112.654495,22.870102,1,1089,佛山市,高明区,1089,佛山市,高明区
...,...,...,...,...,...,...,...,...,...,...,...,...,...
901655,ws06fn3,112.941513,23.018417,ws06fp2,112.940140,23.023911,1,1074,佛山市,南海区,1074,佛山市,南海区
901656,ws06z7n,113.179092,23.000565,ws06z6y,113.179092,22.999191,2,1267,佛山市,南海区,1267,佛山市,南海区
901657,ws073pt,112.903061,23.113174,ws07905,112.900314,23.115921,1,1428,佛山市,三水区,1428,佛山市,三水区
901658,ws0jry3,112.842636,23.457870,ws0jryd,112.844009,23.459244,1,1092,佛山市,三水区,1094,佛山市,三水区


In [17]:
# 保存CSV
home_work_merge_df.to_csv(r'./data/output/3.5职住人口关联分布数据.csv', encoding='utf_8_sig', index=False)
home_work_merge_df.to_json(r'./data/output/3.5职住人口关联分布数据.json', index=False)

### 3.6 职住人口关联统计数据（按区县）

In [18]:

df_3_6 = home_work_merge_df.groupby(['home_district_name',
                                     'work_district_name'])[['count']].sum().reset_index()
df_3_6


,home_district_name,work_district_name,count
0,三水区,三水区,91787
1,三水区,佛山外部,698
2,三水区,南海区,8309
3,三水区,禅城区,930
4,三水区,顺德区,227
5,三水区,高明区,175
6,佛山外部,三水区,461
7,佛山外部,佛山外部,4160
8,佛山外部,南海区,1343
9,佛山外部,禅城区,80


In [19]:
df_3_6.to_csv(r'./data/output/3.6职住人口关联统计_行政区.csv', encoding='utf_8_sig', index=False)
df_3_6.to_json(r'./data/output/3.6职住人口关联统计_行政区.json', index=False)

### 3.7 职住人口关联统计数据（按交通小区）

In [20]:
df_3_7 = home_work_merge_df.groupby(['home_small_zone',
                                     'work_small_zone'])[['count']].sum().reset_index()
df_3_7

,home_small_zone,work_small_zone,count
0,1001,1001,946
1,1001,1002,19
2,1001,1003,10
3,1001,1004,18
4,1001,1005,18
...,...,...,...
67954,8001,8001,222
67955,8002,1094,4
67956,8002,8002,6
67957,21000,1017,1


In [21]:
df_3_7.to_csv(r'./data/output/3.7职住人口关联统计_按交通小区.csv', encoding='utf_8_sig', index=False)
df_3_7.to_json(r'./data/output/3.7职住人口关联统计_按交通小区.json', index=False)

### 3.8 出行OD人口统计数据（按区县）

In [22]:
df = pd.read_pickle(r'./data/input/join/move_join_fix1')
df

,date,is_local,start_half,mode,start_grid_id,end_grid_id,count,last_grid,id_list,label,start_small_zone,start_city_name,start_district_name,start_grid_lon,start_grid_lat,end_small_zone,end_city_name,end_district_name,end_grid_lon,end_grid_lat
0,20230501,NaN,0-0,0,ws04pn6,ws04ppy,1,-1,3,1,1086,佛山市,高明区,112.811050,22.886581,1086,佛山市,高明区,112.816544,22.894821
1,20230501,NaN,13-1,1,ws04qzm,ws04ppy,1,-1,1611748,1,1087,佛山市,高明区,112.804184,22.936020,1086,佛山市,高明区,112.816544,22.894821
2,20230503,Y,19-1,1,ws06227,ws04ppy,1,-1,1890333,1,1087,佛山市,高明区,112.867355,22.897568,1086,佛山市,高明区,112.816544,22.894821
3,20230505,Y,10-1,1,ws0628h,ws04ppy,1,-1,1496467,1,1087,佛山市,高明区,112.879715,22.896194,1086,佛山市,高明区,112.816544,22.894821
4,20230501,NaN,20-1,1,ws0624r,ws04ppy,1,-1,2141046,1,1087,佛山市,高明区,112.861862,22.908554,1086,佛山市,高明区,112.816544,22.894821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29965351,20230505,Y,15-1,1,ws011nj,ws011nj,1,-1,31117917,10,1088,佛山市,高明区,112.551498,22.709427,1088,佛山市,高明区,112.551498,22.709427
29965352,20230505,Y,14-1,1000,ws03fuy,ws03fs3,1,-1,31116170,10,1030,佛山市,南海区,112.981339,22.834396,1030,佛山市,南海区,112.963486,22.831650
29965353,20230505,Y,15-1,1,ws018zu,ws018zu,1,-1,31117919,10,1089,佛山市,高明区,112.539139,22.806931,1089,佛山市,高明区,112.539139,22.806931
29965354,20230505,Y,15-1,1,ws0k1f2,ws0k40q,1,-1,31119493,10,1085,佛山市,三水区,112.929153,23.216171,1085,佛山市,三水区,112.948380,23.205185


In [26]:
date_taz_df = pd.DataFrame()
date_district_df = pd.DataFrame()
for i in range(1, 9):
    with open(fr'./data/input/join/move_join_fix{i}', 'rb') as f:
        df = pickle.load(f)

    _date_taz_df = df.groupby(['date', 'start_small_zone', 'end_small_zone'])[['count']].sum().reset_index(drop=False)
    _date_district_df = df.groupby(['date', 'start_district_name', 'end_district_name'])[['count']].sum().reset_index(drop=False)
    del df
    date_taz_df = pd.concat([date_taz_df, _date_taz_df])
    del _date_taz_df
    date_district_df = pd.concat([date_district_df, _date_district_df])
    del _date_district_df

In [28]:
date_district_agg_df = date_district_df.groupby(['date', 
                                                 'start_district_name',
                                                 'end_district_name'])[['count']].sum().reset_index(drop=False)
date_district_agg_df

,date,start_district_name,end_district_name,count
0,20230501,三水区,三水区,466264
1,20230501,三水区,佛山外部,59123
2,20230501,三水区,南海区,69235
3,20230501,三水区,禅城区,10929
4,20230501,三水区,顺德区,6568
...,...,...,...,...
1111,20230531,高明区,佛山外部,12258
1112,20230531,高明区,南海区,13819
1113,20230531,高明区,禅城区,7432
1114,20230531,高明区,顺德区,3699


In [29]:
date_taz_agg_df = date_taz_df.groupby(['date', 
                                       'start_small_zone',
                                       'end_small_zone'])[['count']].sum().reset_index(drop=False)
date_taz_agg_df

,date,start_small_zone,end_small_zone,count
0,20230501,1001,1001,1847
1,20230501,1001,1002,16
2,20230501,1001,1003,8
3,20230501,1001,1004,97
4,20230501,1001,1005,57
...,...,...,...,...
6764941,20230531,21000,8007,1
6764942,20230531,21000,9002,1
6764943,20230531,21000,14000,1
6764944,20230531,21000,18000,2


In [30]:
date_taz_agg_df.to_json(r'./data/output/3.9_出行OD统计_交通小区.json')
date_district_agg_df.to_json(r'./data/output/3.8_出行OD统计_行政区.json')